# Double-loop Framework Implementation with Fossil Case: Integrated Ultra-supercritical Power Plant Model


Author: Naresh Susarla (naresh.susarla@netl.doe.gov)

This notebook presents an implementation of the double-loop framework for the fossil case study using the integrated ultra-supercritical power plant model with a molten-salt based thermal energy storage. In order to run this notebook, we need multiple packages including, DISPATCHES, IDAES, and gridx-prescient.

For a given integrated energy system, the double loop workflow has six key steps:

 1. Forecasting: Generate plausible price forcasts for the bidding horizon (chosen to be 48 hours).
 2. Bidding: Formulate and solve a multi-period bidding problem for the generated set of forcasted price signals. The solution of the bidding problem yields a time-varying bid curve.
 3. Day Ahead (DA) Market: Using the bid curve generated in the previous step, solve the unit commitment problem for the day ahead market.
 4. Real Time Market: Fix unit commitment decisions and solve the unit commitment problem for the real time market. The solution yields real time local marginal prices and the optimal dispatch profile for each generator.
 5. Tracking: Formulate and solve a tracking problem to track the optimal dispatch profile obtained in the previous step, and minimize under-production/over-production of electricity.
 6. Settlement: Compute all the cash flows

To start the analysis, first import all necessary modules and packages including Prescient simulator and `MultiPeriodUsc` class. `MultiPeriodUsc` builds a multi-period model of the integrated ultra-supercritical power plant model with a molten salt energy storage.

In [1]:
# Import Python library packages
from types import ModuleType
import pandas as pd
from importlib import resources

# Import Pyomo packages
import pyomo.environ as pyo

# Import IDAES packages
from idaes.apps.grid_integration import Tracker
from idaes.apps.grid_integration import Bidder
from idaes.apps.grid_integration.forecaster import Backcaster
from idaes.apps.grid_integration import DoubleLoopCoordinator
from idaes.apps.grid_integration.model_data import ThermalGeneratorModelData

# Import Prescient simulator
from prescient.simulator import Prescient

# Import integrated ultra-supercritical power plant with energy storage model
from dispatches.case_studies.fossil_case.ultra_supercritical_plant.storage.multiperiod_double_loop_usc import MultiPeriodUsc
from dispatches_sample_data import rts_gmlc

Interactive Python mode detected; using default matplotlib backend for plotting.


Following the imports, set up the fossil generator characteristics using a `model_data` object as shown below. The fossil generator used in this analysis is the integrated ultra-supercritical power plant, refered to in this analysis as the fossil-based integrated energy system (IES). A more detailed explanation of the integrated model is given in the notebook `integrated_storage_with_powerplant`. The double-loop model parameters are the number of hours for the bidding and tracking models, number of scenarios to use, and the number of days for simulation. The fossil generator data included in `model_data` and needed during this analysis is given and described in the list below: 

 - gen_name: The ID of the generator in the RTS - GMLC that is being replaced by the fossil-based IES. The chosen generator name `gen_name` is `102_STEAM_3`
 - generator_type: Accepts either `thermal` or `renewable`. We choose `thermal` for both fossil generators
 - p_min: Minimum power output to the grid in MW. The minimum power output to the grid for this IES is 286 MW
 - p_max: Maximum power output to the grid in MW. The maximum power output to the grid for this IES is 460 MW
 - min_down_time: Mininum downtime after the generator is shutdown. The minimum downtime for this IES is 4 hours
 - min_up_time: Minimum uptime after the generator is turned on. The minimum uptime for this IES is 8 hours
 - ramp_up_60min: Ramp up rate in MW/hr. The default ramp up rate for this IES is 60 MW/hr
 - ramp_down_60min: Ramp down rate in MW/hr. The default ramp down rate for this IES is the same as the ramp up rate, 60 MW/hr
 - shutdown_capacity: Power output when shutdown is initiated in MW. For this IES, during shutdown the plant operates at p_min
 - startup_capacity: Power output when the generator is turned on. For this IES, during startup, the plant operates at p_min as considered during shotdown
 - production_cost_bid_pairs: Cost of production of electricity in \\$/MW as function of power output in MW
 - startup_cost_pairs: (Number of hours after shutdown, cost of startup)
 - fixed_commitment: 0/1 indicates that plant shutdown is allowed/not allowed

In [2]:
# Import the historical price data from 'original rts-gmlc run' output folder:
bus_name = "Attlee"

da_lmp_historical = [21.288312, 20.419032, 19.689677, 19.983571, 19.983571, 20.419032,
       21.843871, 23.437857, 18.072549, -0.      , -0.      , -0.      ,
       -0.      , -0.      , -0.      , -0.      , 18.861   , 22.51634 ,
       33.752674, 33.752674, 27.050323, 24.617429, 23.07    , 19.689677]

rt_lmp_historical = [23.07    , 22.968387, 24.617429, 24.617429, 27.050323, 22.51634 ,
       22.492903, 23.657742, 21.843871, -0.      , -0.      , -0.      ,
       -0.      , -0.      , -0.      , -0.      , -0.      , 21.916765,
       24.079344, 23.40518 , 22.683303, 21.916765, 20.244451, 18.861   ]

In [3]:
generator_data = {
    "gen_name": "102_STEAM_3",
    "p_min": 286,
    "p_max": 460,
    "min_down_time": 4,
    "min_up_time": 8,
    "ramp_up_60min": 60,
    "ramp_down_60min": 60,
    "shutdown_capacity": 286,
    "startup_capacity": 286,
    "initial_status": -1,
    "initial_p_output": 0,
    "production_cost_bid_pairs": [
        (286, 22.16602),
        (350, 19.0723),
        (400, 18.29703),
        (430, 17.71727),
        (460, 17.71727),
    ],
    "startup_cost_pairs": [(4, 7355.42), (10, 10488.35), (12, 11383.41)],
    "fixed_commitment": 1,
    "bus": bus_name,
}

model_data = ThermalGeneratorModelData(**generator_data)

Next, we set the following parameters needed for the double loop workflow:
 - tracking_horizon: Length of the tracking horizon. We choose it to be 10 hours
 - bidding_horizon: Length of the bidding horizon. We choose it to be 48 hours
 - n_scenario: Number of scenarios to be considered for the bidding problem. The file `idaes.apps.grid_integration.examples.lmp_forecasts_concat.csv` includes the data for ten scenarios, so we can choose any number between 1 and 10
 - n_tracking_hour: Frequency at which the tracking problem needs to be solved (every hour)
 - num_days: Simulation horizon (number of days). For this demonstration, 2 days is chosen for this simulation

In [4]:
tracking_horizon = 4  # hours
bidding_horizon = 4  # hours
n_scenario = 1  # for bidding
n_tracking_hour = 1  # advance n_tracking_hour (i.e. assume we solve every hour)
num_days = 1

Set up the solver and the forecaster objects.

In [5]:
# Create solver
solver = pyo.SolverFactory("ipopt")
solver.options = {
    "max_iter": 200,
}

# Create forecaster object
forecaster = Backcaster({bus_name: da_lmp_historical}, {bus_name: rt_lmp_historical})

Set up the objects for Tracker, Bidder, and Coordinator.

In [6]:
# Tracker
mp_usc_tracker = MultiPeriodUsc(
    model_data=model_data
)

thermal_tracker = Tracker(
    tracking_model_object=mp_usc_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
    tracking_horizon=tracking_horizon,
)

# Projection Tracker
mp_usc_projection_tracker = MultiPeriodUsc(
    model_data=model_data
)

thermal_projection_tracker = Tracker(
    tracking_model_object=mp_usc_projection_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
    tracking_horizon=tracking_horizon,
)

# Bidder
mp_usc_bidder = MultiPeriodUsc(
    model_data=model_data
)
thermal_bidder = Bidder(
    bidding_model_object=mp_usc_bidder,
    n_scenario=n_scenario,
    solver=solver,
    forecaster=forecaster,
    day_ahead_horizon=bidding_horizon,
    real_time_horizon=tracking_horizon,
)

# Coordinator
coordinator = DoubleLoopCoordinator(
    bidder=thermal_bidder,
    tracker=thermal_tracker,
    projection_tracker=thermal_projection_tracker,
)


[+   0.00] Beginning the formulation of the multiperiod optimization problem.
2022-10-26 16:26:20 [INFO] idaes.apps.grid_integration.multiperiod.multiperiod: ...Constructing the flowsheet model for blocks[0]


INFO:idaes.apps.grid_integration.multiperiod.multiperiod:...Constructing the flowsheet model for blocks[0]


>>> Creating USC model and initialization for each time period
2022-10-26 16:26:23 [INFO] idaes.apps.grid_integration.multiperiod.multiperiod: ...Constructing the flowsheet model for blocks[1]


INFO:idaes.apps.grid_integration.multiperiod.multiperiod:...Constructing the flowsheet model for blocks[1]


>>> Creating USC model and initialization for each time period
2022-10-26 16:26:27 [INFO] idaes.apps.grid_integration.multiperiod.multiperiod: ...Constructing the flowsheet model for blocks[2]


...

INFO:idaes.apps.grid_integration.multiperiod.multiperiod:...Constructing the flowsheet model for blocks[2]


>>> Creating USC model and initialization for each time period
2022-10-26 16:27:12 [INFO] idaes.apps.grid_integration.multiperiod.multiperiod: ...Constructing the flowsheet model for blocks[3]


INFO:idaes.apps.grid_integration.multiperiod.multiperiod:...Constructing the flowsheet model for blocks[3]


>>> Creating USC model and initialization for each time period
[+  13.16] Completed the formulation of the multiperiod optimization problem.
2022-10-26 16:27:14 [WARNING] idaes.apps.grid_integration.multiperiod.multiperiod: Initialization function is not provided. Returning the multiperiod model without initialization.


2022-10-26 16:27:14 [WARNING] idaes.apps.grid_integration.multiperiod.multiperiod: unfix_dof function is not provided. Returning the model without unfixing degrees of freedom


Set up the `plugin_module` needed for the Prescient simulator

In [7]:
class PrescientPluginModule(ModuleType):
    def __init__(self, get_configuration, register_plugins):
        self.get_configuration = get_configuration
        self.register_plugins = register_plugins


plugin_module = PrescientPluginModule(
    get_configuration=coordinator.get_configuration,
    register_plugins=coordinator.register_plugins,
)


**IMPORTANT** Before proceeding with the rest of the notebook:
1. Install the `dispatches-sample-data` package by running `pip install dispatches-sample-data` in the working conda environment

OR

1. Download the RTS-GMLC dataset from [https://github.com/GridMod/RTS-GMLC] (or directly as a zipfile here) to your local machine
2. Update the value of the `rst_gmlc_data_dir` variable in the cell below to the appropriate location of the `RTS_Data/SourceData` subfolder on your local machine

In [8]:
# If installing the dispatches-sample-data
rts_gmlc_data_dir = str(rts_gmlc.source_data_path)

# If downloading the data to the local machine use the following option with appropriate path
rts_gmlc_data_dir = "/projects/gmihybridsys/dguittet/data/RTS-GMLC/RTS_Data/SourceData"


Declare the options for Prescient simulator

In [9]:
options = {
    "data_path": rts_gmlc_data_dir,
    "input_format": "rts-gmlc",
    "simulate_out_of_sample": True,
    "run_sced_with_persistent_forecast_errors": True,
    "output_directory": "bidding_multiperiod_usc",
    "start_date": "2020-07-10",
    "num_days": num_days,
    "sced_horizon": tracking_horizon,
    "ruc_horizon": bidding_horizon,
    "ruc_every_hours": bidding_horizon,
    "compute_market_settlements": True,
    "day_ahead_pricing": "LMP",
    "ruc_mipgap": 0.01,
    "symbolic_solver_labels": True,
    "reserve_factor": 0.0,
    "deterministic_ruc_solver": "gurobi",
    "output_ruc_solutions": True,
    "sced_solver": "gurobi",
    "print_sced": True,
    "enforce_sced_shutdown_ramprate": True,
    "plugin": {
        "doubleloop": {
            "module": plugin_module,
            "bidding_generator": "102_STEAM_3",
        }
    }
}


Run Prescient simulator.

In [10]:
Prescient().simulate(**options)

INFO:gurobipy.gurobipy:Restricted license - for non-production use only - expires 2023-10-25


Initializing simulation...
Did not find reserves.csv; assuming no reserves


/home/dguittet/.conda-envs/hybrid/lib/python3.7/site-packages/pandas/io/parsers/base_parser.py:1055: FutureWarning: 
        Use pd.to_datetime instead.

  return generic_parser(date_parser, *date_cols)


Setting default t0 state in RTS-GMLC parser


Dates to simulate: 2020-07-10 to 2020-07-10
RUC activation hours: 0, 4, 8, 12, 16, 20
Final RUC date: 2020-07-10
Using current day's forecasts for RUC solves
Using persistent forecast error model when projecting demand and renewables in SCED


Extracting scenario to simulate
Ipopt 3.13.2: max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for la